In [ ]:
import os
import numpy as np
import pandas as pd
import h5py
from scipy.stats import pearsonr, spearmanr

In [ ]:
# function to read hdf5 file
def read_hdf5(event, filepath, key):
    if event:
        op = os.path.join(filepath, event+'.hdf5')
    else:
        op = filepath
    
    if os.path.exists(op):
        with h5py.File(op, 'r') as f:
            arr = np.asarray(f[key])
    else:
        raise Exception('{}.hdf5 file does not exist'.format(event))
    
    return arr

In [ ]:
# filepath to the guppy output folder
filepath = '/Users/VENUS/Downloads/FP_Data/Jillian/F337-210610-145110/F337-210610-145110_output_1'

# filenames of the z-scores traces
filenames = [
    'z_score_D2.hdf5',
    'z_score_DA.hdf5',
    'z_score_Re.hdf5'
]
new_filenames = filenames[1:] + [filenames[0]]

result, index, cols = [], [], []
for i in range(len(filenames)):
    # reading z-score files
    x = read_hdf5('', os.path.join(filepath, filenames[i]), 'data')
    y = read_hdf5('', os.path.join(filepath, new_filenames[i]), 'data')
    
    # check x and y length
    if x.shape[0]>y.shape[0]:
        x = x[:y.shape[0]]
    elif x.shape[0]<y.shape[0]:
        y = y[:x.shape[0]]
    else:
        x, y = x, y
    
    region_a = os.path.splitext(filenames[i])[0].split('_')[-1]
    region_b = os.path.splitext(new_filenames[i])[0].split('_')[-1]
    cols.append(['{}_{}_{}'.format(region_a, region_b, 'r'), '{}_{}_{}'.format(region_a, region_b, 'p')])
    
    # running parammetric pearson correlation test
    r, p = pearsonr(x, y)
    result.append([r, p])
    index.append('{}_{}_{}'.format(region_a, region_b, 'pearsonr'))

    
    # running non-parametric spearman correlation test
    r_np, p_np = spearmanr(x, y, axis=None)
    result.append([r_np, p_np])
    index.append('{}_{}_{}'.format(region_a, region_b, 'spearmanr'))

# saving to a csv file
df = pd.DataFrame(result, index=index, columns=['r-value', 'p-value'])
df.to_csv(os.path.join(filepath, 'z_score_corr.csv'))